In [15]:
import numpy as np
import os
from pathlib import Path
import pickle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input


# Feature extraction
We will use the VGG16 network as a signal processor, generating a feature descriptor for each image that we can use later for classification.

Get the weights of the VGG16 model

In [2]:
vgg16_path = Path('..','models','VGG16.h5')
if not vgg16_path.is_file():
    vgg16 = keras.applications.VGG16(include_top=True,  # include fully connected layers
                                     weights='imagenet') # use pre-trained model
    vgg16.save(vgg16_path)
    
else:   
    vgg16 = keras.models.load_model(vgg16_path)



We can see the strutcure of the VGG16 model here.

In [3]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

The pre-trained model will run data through the entire network and return the output of the classification layer. 
Howevever, we only want the output of the intermediate layer so that we can use it as a feature descriptor. 

In [4]:
def layer_extractor(model=vgg16, layer='fc1'):
    """
    returns a model that will extract the outputs of *layer* from *model*.
    
    Parameters
    -------------
    model: keras model
        full model from which intermediate layer will be extracted
    layer: string
        name of layer from which to extract outputs
    
    Returns
    -------------
    new_model: keras model
        feature extractor model which takes the same inputs as *model* and returns the outputs
        of the intermediate layer specified by *layer* by calling new_model.predict(inputs)
    """
    assert layer in [x.name for x in model.layers]
    new_model = keras.Model(inputs = vgg16.input, outputs=[vgg16.get_layer(layer).output])
    return new_model


fc1_extractor = layer_extractor()

Next, we need to get the file paths of the pre-processed images we saved in 01_preprocess.ipynb. 

In [5]:
img_root = Path('..','data','images_preprocessed','images_histeq_resize')
assert img_root.is_dir()
files = sorted(img_root.glob("*.bmp"))

## Shuffle the filenames so they appear randomly in the dataset
rs = np.random.RandomState(seed=749976)
rs.shuffle(files)

assert len(files) == 1800

For feature extraction to work correctly, the images have to be in the correct format for the network weights.
Keras gives us functions for loading and formatting these images. Note the function is called 'preprocessing,'
but it does not actually change the properties of the image like the preprocessing we did before. Instead, it 
ensures that the images are represented the correct way.

In [6]:
images = [image.load_img(file) for file in files] # load images
images = np.asarray([image.img_to_array(img) for img in images]) # convert images to an array with shape consistent for the vgg16 input
images = preprocess_input(images) # normalizes the pixel values to match the imagenet format (and therefore the pre-trained weights)
print(images.shape) # 1800 images, 224x224 pixels, 3 color channels

(1800, 224, 224, 3)


Now we are ready to extract the features. This will take several minutes to run, there are a lot of operations to do!

In [7]:
fc1 = fc1_extractor.predict(images)
print(fc1.shape)

(1800, 4096)


Now we are going to save the results

It is convenient to extract the labels here as well. The labels are determined from the characteris in the filename before the first "_".

In [8]:
def extract_labels(f): return [x.stem.split('_')[0] for x in f]
labels = extract_labels(files)

In [9]:
results = {'filename' : files,
           'features': fc1,
          'labels': labels,
           'layer_name': 'fc1'
          }

feature_dir = Path('..','data','features')
os.makedirs(feature_dir, exist_ok=True)
with open(feature_dir / 'VGG16_fc1_features_std.pickle', 'wb') as f:
    pickle.dump(results, f)

# Label encoding
One last step that will make our lives easier throughout the analysis is standardizing the
encoding of labels. The labels are stored as strings in the filenames, but it will be more
convenient to convert them to numeric values so we can do math on them.
We can create one LabelEncoder model and save it for reuse throughout the study.

In [16]:
labels_unique = sorted(np.unique(labels),
                       key = lambda x: x.upper()) # removes effect of capitalization on sorting
le = LabelEncoder()
le.fit(labels_unique)
labels_int = le.transform(labels[:10])
labels_str = le.inverse_transform(labels_int)

print('labels_unique: {}'.format(labels_unique))
print('first 10 integer labels: {}'.format(labels_int))
print('first 10 string labels: {}'.format(labels_str))

labels_unique: ['Cr', 'In', 'Pa', 'PS', 'RS', 'Sc']
first 10 integer labels: [3 4 5 1 0 0 4 1 0 4]
first 10 string labels: ['Pa' 'RS' 'Sc' 'In' 'Cr' 'Cr' 'RS' 'In' 'Cr' 'RS']


In [21]:
with open(Path('..','models','label_encoder.pickle'), 'wb') as f:
    pickle.dump(le, f)

# Other feature representations
As part of the sensitivity analysis we will need some other feature representations as well, to see how these changes impact the cluster analysis.

## No histogram equalization

In [10]:
img_root_nohisteq = Path('..','data','images_preprocessed','images_resize')
assert img_root_nohisteq.is_dir()
files_noh = sorted(img_root_nohisteq.glob('*'))

rs = np.random.RandomState(seed=3626210179)

rs.shuffle(files_noh)

assert len(files_noh) == 1800

In [11]:
# follow the same process described above to load images, convert to array, and format for vgg16

images_noh = [image.load_img(file) for file in files_noh]
images_noh = np.asarray([image.img_to_array(img) for img in images_noh])
images_noh = preprocess_input(images_noh)

print(images_noh.shape)



(1800, 224, 224, 3)


In [12]:
fc1_extractor = layer_extractor()
fc1_noh = fc1_extractor.predict(images_noh)

print(fc1_noh.shape)

(1800, 4096)


In [13]:
labels_noh = extract_labels(files_noh)
results = {'filename': files_noh,
           'features': fc1_noh,
           'labels' : labels_noh,
           'layer_name': 'fc1 (no histeq)'}

feature_dir = Path('..','data','features')
with open(feature_dir / 'VGG16_fc1_features_NoHistEQ.pickle', 'wb') as f:
    pickle.dump(results, f)